## Instalasi Library

In [9]:
#Untuk mengakses dataset dari Kaggle melalui API
!pip install -q kaggle

#Untuk membuat antarmuka pengguna berbasis web agar bisa upload dan prediksi secara interaktif.
!pip install -q streamlit

#Untuk membuat tunnel agar aplikasi Streamlit yang dijalankan di Colab dapat diakses dari internet.
!pip install -q pyngrok

#Untuk menangani proses resampling audio
!pip install -q resampy

#Untuk ekstraksi fitur audio dan tensorflow untuk melatih model deep learning (CNN).
!pip install -q librosa soundfile numpy tensorflow


## Upload `kaggle.json` untuk akses dataset RAVDESS

In [1]:
# Mengunggah file kaggle.json yang berisi API key untuk mengakses dataset dari Kaggle.
from google.colab import files
files.upload()


{}

In [11]:
# Mengakses dataset voice untuk di trained.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio
!unzip -q ravdess-emotional-speech-audio.zip -d ravdess


Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
 99% 424M/429M [00:02<00:00, 203MB/s]
100% 429M/429M [00:02<00:00, 219MB/s]


## Mount Google Drive dan load model pre-trained

In [12]:
# Menghubungkan Google Colab dengan Google Drive untuk mengakses model cnn
from google.colab import drive
drive.mount('/content/drive')

model_path = '/Masukan akses link google drive untuk mengkases model trained'


Mounted at /content/drive


## Tulis Streamlit app ke `app.py`

In [27]:
%%writefile app.py
import streamlit as st
import librosa
import numpy as np
import tensorflow as tf
from keras.models import load_model

# ====================
# Load CNN model
# ====================
model = load_model("model_ravdess_cnn.h5")  # Pastikan file ini ada di /content atau path yang benar

# Map dari indeks ke label emosi
emotion_map = {
    0: 'neutral', 1: 'calm', 2: 'happy', 3: 'sad',
    4: 'angry', 5: 'fearful', 6: 'disgust', 7: 'surprised'
}

# ====================
# Fungsi Ekstraksi Fitur
# ====================
def extract_features(file, max_pad_len=216):
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast')

    # Ekstrak 40 MFCC
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

    # Padding agar panjang waktu selalu 216
    pad_width = max_pad_len - mfccs.shape[1]
    if pad_width > 0:
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_pad_len]

    # Bentuk akhir untuk CNN: (40, 216, 1)
    mfccs = mfccs[..., np.newaxis]

    return mfccs

# ====================
# UI Streamlit
# ====================
st.title("Prediksi Emosi dari Suara")
st.write("Upload file audio (.wav) untuk melihat prediksi emosi")

uploaded_file = st.file_uploader("Upload file .wav", type=["wav"])

if uploaded_file is not None:
    # Tampilkan player audio
    st.audio(uploaded_file, format="audio/wav")

    # Ekstrak fitur dari audio dan ubah ke shape CNN
    features = extract_features(uploaded_file)           # (40, 216, 1)
    features = np.expand_dims(features, axis=0)          # (1, 40, 216, 1)

    # Prediksi
    prediction = model.predict(features)
    emotion_index = np.argmax(prediction)
    emotion = emotion_map[emotion_index]

    # Tampilkan hasil prediksi
    st.success(f"Prediksi Emosi: **{emotion.capitalize()}**")


Overwriting app.py


In [30]:
# Trained model
!pip install -q resampy
!python "Masukan link google drive model"

ERROR: Operation cancelled by user


KeyboardInterrupt: 

## Jalankan Streamlit App

In [25]:
from pyngrok import ngrok
!ngrok config add-authtoken # Masukan token key ngrok.

# Stop ngrok
!killall ngrok

!streamlit run app.py &>/content/log.txt &

url = ngrok.connect(addr=8501)
print("🌍 Buka aplikasi kamu di sini:", url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
ngrok: no process found
🌍 Buka aplikasi kamu di sini: NgrokTunnel: "https://da41-34-169-210-16.ngrok-free.app" -> "http://localhost:8501"


In [24]:
# Mengecek ukuran model
!ls -lh model_ravdess_cnn.h5

-rw-r--r-- 1 root root 61M Jun 18 07:41 model_ravdess_cnn.h5
